In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
from matplotlib import rc
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.pipeline import Pipeline
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import silhouette_score, mean_squared_error
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram
import time

from sklearn import model_selection
from sklearn.feature_selection import SelectFromModel
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor, NearestNeighbors
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score, f1_score

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

/home/luke/anaconda2/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/luke/anaconda2/envs/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
%matplotlib inline
pd.set_option('max_columns',500)
font = {'size': 20}
rc('font', **font)
plt.style.use('seaborn-bright')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
df = pd.read_pickle('data/SRP/clean_data_public_no_crime_lag2_by_store.pkl')

In [ ]:
cust_table_clust = pd.read_csv('data/SRP/cust_table.pkl')
df = df.join(cust_table_clust, on='address1', how='left')

# Split into features vs targets

In [5]:
def X_y(df, non_feature_cols, target_col):
    non_feature_data = df[non_feature_cols]
    features = list(set(df) - set(non_feature_cols))
    features.sort()
    X = df[features]
    y = non_feature_data[target_col]
    return X, y

In [6]:
def time_split(df, date_col, date, non_feature_cols, target_col):
    df_train = df[ df[date_col] < date ]
    df_test = df[ df[date_col] >= date ]
    X_train, y_train = X_y(df_train, non_feature_cols, target_col)
    X_test, y_test = X_y(df_test, non_feature_cols, target_col)
    return X_train, X_test, y_train, y_test

In [7]:
# split into features and targets
non_feature_cols = ['shrink_value', 'shrink_to_sales_value_pct', 'shrink_value_out', 'shrink_to_sales_value_pct_out',
               'shrink_value_ex_del', 'shrink_to_sales_value_pct_ex_del', 'qty_inv_out', 'qty_shrink',
               'qty_shrink_ex_del', 'qty_shrink_out', 'qty_end_inventory', 'qty_f', 'qty_out', 'qty_ex_del',
               'qty_n', 'qty_delivery', 'qty_o', 'qty_d', 'qty_shrink_per_day', 'shrink_value_per_day']
X, y = X_y(df, non_feature_cols, target_col='shrink_value_per_day')
# del df # free up memory

In [8]:
# random split for model testing (try/except based on customer seg being completed)
try:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=X.cluster.values)
except:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# time split for forcast
split_date = pd.to_datetime('12/01/2017')
forc_X_train, forc_X_test, forc_y_train, forc_y_test = time_split(df, 'visit_date', split_date, 
                                                                  non_feature_cols, target_col='shrink_value_per_day')

# Standardize and Scale

In [9]:
def std_f(X_std):
    std_mask = (X_std.dtypes == int) | (X_std.dtypes == np.float64) # only standardize numbers that are not associated with time features
    std_cols = X_std.columns[std_mask]
    ss = StandardScaler()
    X_std[std_cols] = ss.fit_transform(X_std[std_cols])
    return X_std

In [10]:
def scale_f(X_sc):
    sc_mask = (X_sc.dtypes == np.float32) # only scale time features
    sc_cols = X_sc.columns[sc_mask]
    min_time = X_sc[sc_cols].min().values.min()
    max_time = X_sc[sc_cols].max().values.max()
    for col in sc_cols:
        # scale all time features using the same two values, so equivalent values reference the same date across columns
        X_sc[col] = (X_sc[col] - min_time) / (max_time - min_time)
    return X_sc

In [11]:
def ss(X_ss, std=True, scale=True):
    if not std and not scale:
        return
    X_new = X_ss.copy()
    if std:
        X_new = std_f(X_new)
    if scale:
        X_new = scale_f(X_new)
    return X_new

In [12]:
# standardize and scale data
X_train = ss(X_train, std=True, scale=True)
X_test = ss(X_test, std=True, scale=True)
forc_X_train = ss(forc_X_train, std=True, scale=True)
forc_X_test = ss(forc_X_test, std=True, scale=True)

In [13]:
# create mask of all numberical columns to be used in clustering/modeling

# including time features
numb_mask = (X_train.dtypes == int) | (X_train.dtypes == np.float64) | (X_train.dtypes == np.float32) | (X_train.dtypes == np.uint8)
numb_cols = X_train.columns[numb_mask]

# not including time features
numb_no_time_mask = (X_train.dtypes == int) | (X_train.dtypes == np.float64) | (X_train.dtypes == np.uint8)
numb_no_time_cols = X_train.columns[numb_no_time_mask]

# forcasting columns (what is known months ahead of time)
forc_cols = ['FD_ratio', 'LAPOP1_10', 'POP2010', 'customer_id_1635139',
             'customer_id_1903139', 'customer_id_2139', 'customer_id_2331150',
             'customer_id_2741156', 'customer_id_2773156', 'customer_id_2782156',
             'customer_id_2956160', 'customer_id_2977160', 'customer_id_3083182',
             'customer_id_3088198', 'customer_id_3088201', 'customer_id_3089336',
             'customer_id_3093327', 'customer_id_3093329', 'customer_id_3097348',
             'dens_sq_mile', 'shrink_value_per_day_lag1', 'shrink_value_per_day_lag2', 'unemp_rate']
# mask to be used in calculations
model_mask_cols = numb_no_time_cols

# Feature Importance
    - PCA
    - SVD
    - Random forest

## PCA

In [ ]:
def plot_var(pca):
    '''
    Input: fitted PCA
    '''
    var_arr = np.insert(pca.explained_variance_ratio_, [0], 0)
    cum_arr = np.cumsum(var_arr)
    feat_arr = np.arange(0, len(var_arr), 1)
    
    plt.figure(figsize=(12,8))
    plt.plot(feat_arr, var_arr, label='Variance at each point', c='r')
    plt.plot(feat_arr, cum_arr, label='Cumulative variance')
    plt.grid(alpha=0.3)
    plt.legend()
    plt.ylabel('Fraction of total variance explained')
    plt.xlabel('Principal Component')
    plt.xticks(feat_arr);

In [ ]:
pca = PCA(10)
pca.fit(X_train[model_mask_cols])
plot_var(pca)

In [ ]:
pca.components_

## SVD

In [ ]:
svd = TruncatedSVD(n_components=10)
svd.fit(X_train[model_mask_cols])
plot_var(svd)

# Clustering Models
Clustering models to try:
    - k-means
    - Heirarchal clustering
        - Look into the neat visualizations in R!
Link [here](http://www.sthda.com/english/wiki/beautiful-dendrogram-visualizations-in-r-5-must-known-methods-unsupervised-machine-learning)
    - Neural net
Perform for both subjective (features only) and objective (including targets/possibly even only targets)

## Customer Profile

In [ ]:
# df['theft/person'] = df['2015_theft_count'] / df['POP2010']

In [ ]:
df.head()

In [ ]:
cust_table = df.groupby(['address1']).mean()[['qty_shrink_per_day', 'shrink_value_per_day', 'POP2010',
                                              'FD_ratio', 'unemp_rate', 'dens_sq_mile', ]].reset_index()
cust_table.set_index('address1', inplace=True)

city_i = df.columns.get_loc('city')
state_i = df.columns.get_loc('state')
zip_i = df.columns.get_loc('zip_code')
cust_i = df.columns.get_loc('customer_id')
for index, row in cust_table.iterrows():
    foo = df[ df.address1 == index]
    for i, r in foo.iterrows():
        city = r[city_i]
        state = r[state_i]
        zip_code = r[zip_i]
        cust_id = r[cust_i]
        
        cust_table.set_value(index, 'city', city)
        cust_table.set_value(index, 'state', state)
        cust_table.set_value(index, 'zip_code', zip_code)
        cust_table.set_value(index, 'customer_id', cust_id)
        break

/home/luke/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/luke/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/luke/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/home/luke/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [ ]:
cust_table.head()

In [ ]:
dummy_cust = pd.get_dummies(cust_table, columns=['customer_id','zip_code'])
# including shrink and not inluding dummies
shrink_cust_mask = (dummy_cust.dtypes == float)
shrink_cust_cols = dummy_cust.columns[shrink_cust_mask]

# including dummies but not shrink
dummy_cust_mask = (dummy_cust.dtypes == float) | (dummy_cust.dtypes == np.uint8)
dummy_cust_cols = dummy_cust.columns[dummy_cust_mask]
dummy_cust_cols = list(dummy_cust_cols)
dummy_cust_cols.remove('qty_shrink_per_day')
dummy_cust_cols.remove('shrink_value_per_day')

# including dummies and shrink
all_cust_mask = (dummy_cust.dtypes == float) | (dummy_cust.dtypes == np.uint8)
all_cust_cols = dummy_cust.columns[dummy_cust_mask]

std_cust = std_f(dummy_cust.copy())

### Feature Importance

In [ ]:
print('Shape before regularization: ',std_cust[dummy_cust_cols].shape)
lasso = Lasso(alpha=0.01)
lasso.fit(std_cust[dummy_cust_cols], std_cust['shrink_value_per_day'])
model = SelectFromModel(lasso, prefit=True)
std_cust_reduc = model.transform(std_cust[dummy_cust_cols])
print('Shape after regularization: ',std_cust_reduc.shape)
std_cust_reduc

In [ ]:
print('Shape before regularization: ',std_cust[dummy_cust_cols].shape)
lsvr = LinearSVR(C=0.01, loss='epsilon_insensitive', dual=True)
lsvr.fit(std_cust[dummy_cust_cols], std_cust['shrink_value_per_day'])
model = SelectFromModel(lsvr, prefit=True)
std_cust_reduc = model.transform(std_cust[dummy_cust_cols])
print('Shape after regularization: ',std_cust_reduc.shape)
std_cust_reduc
model.get_support

In [ ]:
dummy_cust_cols

### Customer Clustering

In [ ]:
# columns to use in segmentation:
pca_cols = ['qty_shrink_per_day', 'shrink_value_per_day', 'FD_ratio', 'dens_sq_mile', 'POP2010', 'unemp_rate']
clusters = np.arange(1, 15)
SSE_arr, ss_arr = kmeans(std_cust[all_cust_cols], clusters)
#elbow(clusters, SSE_arr)
silhouette(np.arange(2, 15), ss_arr)

In [ ]:
cust_kmeans = KMeans(n_clusters=5, max_iter=10000, n_jobs=-1)
pred = cust_kmeans.fit_predict(std_cust[all_cust_cols])
dummy_cust['cluster'] = pred

In [ ]:
dummy_cust.head()

In [ ]:
print(dummy_cust.groupby('cluster').count().city)
dummy_cust.groupby('cluster').mean()[all_cust_cols]

In [ ]:
cust_pca = PCA(2)
pcas = cust_pca.fit_transform(std_cust[all_cust_cols])

plt.figure(figsize=(12,12))
plt.scatter(pcas[:,0], pcas[:,1], c=dummy_cust.cluster)

In [ ]:
cust_pca = PCA(3)
pcas = cust_pca.fit_transform(std_cust[all_cust_cols])

fig = plt.figure(figsize=(12,12))
ax = Axes3D(fig)
ax.scatter(pcas[:,0], pcas[:,1], pcas[:,2], s=20, alpha=1, c=dummy_cust.cluster)
ax.set_xlim(left=-10, right=2)
ax.set_ylim(bottom=0, top=10)
ax.set_zlim(top=5)

In [ ]:
#cust_table_clust = cust_table[['cluster']].astype(str)
cust_table_clust = dummy_cust[['cluster']].astype(str)
cust_table_clust.info()

## K-means

In [24]:
def kmeans(X_km, clusters):
    SSE_arr = []
    ss_arr = []
    for i in clusters:
        kmeans = KMeans(n_clusters=i, n_jobs=-1)
        clust_dist = kmeans.fit_transform(X_km)
        clust_num = kmeans.predict(X_km)

        SSE = 0
        for a, b in zip(clust_dist, clust_num):
            SSE += a[b] ** 2
        SSE_arr.append(SSE)
        
        if i > 1:
            ss_arr.append(silhouette_score(X_km, clust_num))
    return SSE_arr, ss_arr

In [ ]:
clusters = np.arange(1, 20)
SSE_arr, ss_arr = kmeans(X_train[model_mask_cols], clusters)
elbow(clusters, SSE_arr)
silhouette(np.arange(2, 20), ss_arr)

In [26]:
def elbow(clusters, SSE_arr):
    plt.figure(figsize=(12,8))
    plt.title('Elbow Plot')
    plt.plot(clusters, SSE_arr)
    plt.grid(alpha=0.3)
    plt.xticks(clusters)
    plt.xlabel('Number of Clusters')
    plt.ylabel('Sum of Squares Error (SSE)');

In [27]:
def silhouette(clusters, ss_arr):
    plt.figure(figsize=(12,8))
    plt.title('Silhouette Scores')
    plt.plot(clusters, ss_arr)
    plt.grid(alpha=0.3)
    plt.xticks(clusters)
    plt.xlabel('Number of Clusters')
    plt.ylabel('Silhouette Score');

## Heirarchal Clustering

In [ ]:
def heir_clust(X_hc, thresh, dist_metric='cosine', num_params_to_display=50):
    # Find distances using pair-wise distances in the array, according to desired metric
    dist = squareform(pdist(X_hc.values.T, metric = dist_metric))

    # Plot dendrogram
    fig, axarr = plt.subplots(nrows = 3, ncols = 1, figsize=(60, 80))
    for ax, linkmethod in zip(axarr.flatten(), ['single', 'complete', 'average']):
        clust = linkage(dist, method=linkmethod)
        dendrogram(clust, ax=ax, truncate_mode='lastp', p=num_params_to_display, labels=model_mask_cols, 
                   color_threshold=thresh, leaf_font_size=25) #color threshold number sets the color change
        ax.set_title('{} linkage'.format(linkmethod), fontsize=40)
        ax.grid(alpha=0.3)
    plt.savefig('images/clust.png'.format(linkmethod))

In [ ]:
heir_clust(X_train[model_mask_cols], thresh=1.6)

# Regression Models
Regression models to try:
    - Linear regression (with additional complexity)
    - Random forest
    - Boosting
    - Gradient descent
    - Neural net

## OLS

In [ ]:
def ols(X_train, X_test, y_train, y_test):
    ols = OLS(y_train, add_constant(X_train.values, has_constant='add'))
    result = ols.fit()
    pred = result.predict(add_constant(X_test.values, has_constant='add'))
    score = mean_squared_error(y_test, pred)
    print('Root Mean Square Error: ',score)
    names = list(X_train.columns)
    names.insert(0,'Constant')
    print(result.summary(xname=names))

In [ ]:
ols(X_train[model_mask_cols], X_test[model_mask_cols], y_train, y_test)

In [ ]:
for val in X_train.item_category.unique():
    print('Item cat: ', val)
    mask = X['item_category_{}'.format(val)] == 1
    X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(X[mask], y[mask], test_size=0.2)
    size = len(X_train_temp)
    print('Size: ', size)
    if size < 30:
        continue
    ols(X_train_temp[model_mask_cols], X_test_temp[model_mask_cols], y_train_temp, y_test_temp)

## multiple Sklear models

In [ ]:
def class_crossval(X, y, models, scoring='neg_mean_absolute_error'):
    results = []
    names = []
    all_scores = []
    print('Mod - Avg - Std Dev')
    print('---   ---   -------')
    for name, model in models:
        kfold = model_selection.KFold(n_splits=10, random_state=seed)
        cv_results = model_selection.cross_val_score(model, X, y, cv=kfold, scoring=scoring, n_jobs=-1)
        results.append(cv_results)
        names.append(name)
        print('{}: {:.2f} ({:2f})'.format(name, cv_results.mean(), cv_results.std()))
    
    fig = plt.figure(figsize=(25, 18))
    plt.tight_layout()
    fig.suptitle('Algorithm Comparison of CrossVal Scores')
    ax = fig.add_subplot(111)
    sb.violinplot(data=results, orient='v')
    ax.set_xticklabels(names, rotation=45, ha='right')
    ax.set_ylabel('K-Fold CV Negative Mean Abs. Error')
    ax.set_xlabel('Model')
    plt.grid(alpha=0.4)
    #plt.savefig('images/model_selection_shrink_value.png')

In [ ]:
# Initial Cross Validation
models = []
models.append(('Linear Regression', LinearRegression()))
models.append(('Ridge Regression', Ridge()))
models.append(('Lasso Regression', Lasso()))
models.append(('Elastic Net', ElasticNet()))
#models.append(('Stochastic Gradient Descent', SGDRegressor(max_iter=10000, tol=0.001)))
#models.append(('Support Vector Regression', SVR(max_iter=10000)))
models.append(('K Nearest Neighbors', KNeighborsRegressor(n_jobs=-1)))
models.append(('Decision Tree', DecisionTreeRegressor()))
models.append(('Random Forest', RandomForestRegressor()))
#models.append(('AdaBoost', AdaBoostRegressor(n_estimators=100)))
models.append(('Gradient Boost', GradientBoostingRegressor()))
models.append(('Multi-Layer Perceptron', MLPRegressor(alpha=1)))

class_crossval(X_train[model_mask_cols], y_train, models)

## Random Forest

In [ ]:
def model_grid_plus_error(model, param_grid, X_train, X_test, y_train, y_test):
    test_model = model
    grid = GridSearchCV(test_model, param_grid=param_grid, verbose=1)
    grid.fit(X_train, y_train)
    best_params = grid.best_params_
    model.set_params(**best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return grid, mean_squared_error(y_test, y_pred)

In [ ]:
model = RandomForestRegressor()
params = {'n_estimators': [10, 30], 'max_features': [5, 10, 15], 'max_depth': [None, 20], 'n_jobs': [-1]}
grid, mse = model_grid_plus_error(model, params, X_train[model_mask_cols], X_test[model_mask_cols], y_train, y_test)
print(mse)

## MLP

In [ ]:
model = MLPRegressor()
params = {'hidden_layer_sizes': [(50,50)], 'alpha': [0.000001, 0.00001, 0.0001, 0.001], 'max_iter': [100]}
grid, mse = model_grid_plus_error(model, params, X_train[model_mask_cols], X_test[model_mask_cols], y_train, y_test)
print(mse)
print(grid.best_params_)

# Splitting By Cluster

In [ ]:
value_dist = df.shrink_value_per_day.values
plt.figure(figsize=(10,8))
plt.hist(value_dist, bins=2000)
#plt.yscale('log')
plt.xlim(xmin=-5, xmax=10)
plt.xlabel('Shrink Value/Day')

print('Average: ', value_dist.mean())
print('Std. Dev.: ',value_dist.std())

In [ ]:
X_train.groupby('cluster').count()

In [ ]:
def model_clusters(model, X_train, X_test, col_mask, y_train, y_test):
    cluster_rmse = []
    cluster_models = []
    for clust in range(0, len(X_train.cluster.unique())):
        print(clust)
        train_clust_mask = X_train.cluster == str(clust)
        test_clust_mask = X_test.cluster == str(clust)
        clust_model = model
        clust_model.fit(X_train[col_mask][train_clust_mask], y_train[train_clust_mask])
        y_pred = clust_model.predict(X_test[col_mask][test_clust_mask])
        cluster_rmse.append(np.sqrt(mean_squared_error(y_test[test_clust_mask], y_pred)))
        cluster_models.append(clust_model)
    return cluster_rmse, cluster_models

In [ ]:
cluster_rmse, cluster_models = model_clusters(MLPRegressor(alpha=0.00001, hidden_layer_sizes=(50,50)),  
                                             X_train, X_test, model_mask_cols,y_train, y_test)

In [ ]:
plt.figure(figsize=(8,8))
plt.bar(np.arange(0,len(X_train.cluster.unique())), cluster_rmse)
plt.xlabel('Cluster')
plt.ylabel('Test RMSE')
plt.grid(alpha=0.4)

# Forcasting

What model should look like:
1. Take grouped row for one store on one date:
    - Run through that store's model to predict shrink_value on that date
    

In [14]:
model_mask_cols

Index(['FD_ratio', 'LAPOP1_10', 'POP2010', 'customer_id_1635139',
       'customer_id_1903139', 'customer_id_2139', 'customer_id_2331150',
       'customer_id_2741156', 'customer_id_2773156', 'customer_id_2782156',
       'customer_id_2956160', 'customer_id_2977160', 'customer_id_3083182',
       'customer_id_3088198', 'customer_id_3088201', 'customer_id_3089336',
       'customer_id_3093327', 'customer_id_3093329', 'customer_id_3097348',
       'dens_sq_mile', 'item_category_10', 'item_category_16',
       'item_category_19', 'item_category_26', 'item_category_31',
       'item_category_38', 'item_category_41', 'item_category_43',
       'item_category_44', 'item_category_46', 'item_category_58',
       'item_category_62', 'item_category_77', 'item_category_79',
       'item_category_8', 'item_category_90', 'qty_POG_limit',
       'qty_prev_end_inventory', 'qty_sales',
       'qty_shrink_per_day_lag1_by_store', 'qty_shrink_per_day_lag2_by_store',
       'qty_start_inventory', 'sales_v

In [ ]:
forc_X_train.groupby(['address1', 'visit_date']).mean()

In [ ]:
forc_X_test.groupby(['address1', 'visit_date']).mean()

## LSTM

In [ ]:
def fit_lstm(X, y, batch_size, nb_epoch, neurons):
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
        model.reset_states()
    return model
 
# make a one-step forecast
def forecast_lstm(model, batch_size, X):
    X = X.reshape(1, 1, len(X))
    yhat = model.predict(X, batch_size=batch_size)
    return yhat[0,0]
 
# load dataset
series = read_csv('shampoo-sales.csv', header=0, parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)
 
# transform data to be stationary
raw_values = series.values
diff_values = difference(raw_values, 1)
 
# transform data to be supervised learning
supervised = timeseries_to_supervised(diff_values, 1)
supervised_values = supervised.values
 
# split data into train and test-sets
train, test = supervised_values[0:-12], supervised_values[-12:]
 
# transform the scale of the data
scaler, train_scaled, test_scaled = scale(train, test)
 
# repeat experiment
repeats = 1
error_scores = list()
for r in range(repeats):
    # fit the model
    lstm_model = fit_lstm(X_train[model_mask_cols], y_train, 1, 3, 4)
    # forecast the entire training dataset to build up state for forecasting
    train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)
    lstm_model.predict(train_reshaped, batch_size=1)
    # walk-forward validation on the test data
    predictions = list()
    for i in range(len(test_scaled)):
        # make one-step forecast
        X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
        yhat = forecast_lstm(lstm_model, 1, X)
        # invert scaling
        yhat = invert_scale(scaler, X, yhat)
        # invert differencing
        yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
        # store forecast
        predictions.append(yhat)
    # report performance
    rmse = sqrt(mean_squared_error(raw_values[-12:], predictions))
    print('%d) Test RMSE: %.3f' % (r+1, rmse))
    error_scores.append(rmse)
 
# summarize results
results = pd.DataFrame()
results['rmse'] = error_scores
print(results.describe())
results.boxplot()
pyplot.show()

# Forcasting/Time Series
 - Inclusion of endog (target) variable into predictive model/forcast
 - Explore various techniques outlined statsmodels.pdf
 - Try LSTM neural net

In [ ]:
plt.scatter(X_train.prev_item_move_date_int.values, X_train.prev_visit_date_int.values, alpha=0.01)

In [ ]:
foo = df[ df.address1 == 'SPEEDWAY #1224']
plt.figure(figsize=(12,12))
plt.scatter(foo.visit_date.values, foo.qty_shrink.values, c=foo.item_UPC.values)
plt.xticks(rotation=45)
plt.legend()

In [ ]:
def cat_plot_sales():
    freq = '2w'
    item_filt = df.groupby(['item_category', pd.Grouper(key='visit_date', freq=freq)]).mean().reset_index()
    fig = plt.figure(figsize=(12,12))
    i = 1
    for cat in item_filt.item_category.unique():
        if cat == '41':
            continue
        foo = item_filt[ item_filt.item_category == cat]
        ax = fig.add_subplot(1,1,1)
        ax.plot(foo.visit_date, foo.sales_value, label=cat)
        ax.set_xticklabels(foo.visit_date, rotation=45, ha='right')
    plt.legend()
    plt.ylabel('Average Lost Sales/{} ($)'.format(freq))
    plt.xlabel('Date')
    plt.grid(alpha=0.3)
    plt.title('Average Sales Loss Across All Stores by Cat')
    
cat_plot_sales()

In [ ]:
def cat_plot_shrink():
    freq = '2w'
    item_filt = df.groupby(['item_category', pd.Grouper(key='visit_date', freq=freq)]).mean().reset_index()
    fig = plt.figure(figsize=(12,12))
    i = 1
    for cat in item_filt.item_category.unique():
        if cat == '41':
            continue
        foo = item_filt[ item_filt.item_category == cat]
        ax = fig.add_subplot(1,1,1)
        ax.plot(foo.visit_date, foo.qty_shrink, label=cat)
        ax.set_xticklabels(foo.visit_date, rotation=45, ha='right')
    plt.legend()
    plt.ylabel('Average Shrink/{} ($)'.format(freq))
    plt.xlabel('Date')
    plt.grid(alpha=0.3)
    plt.title('Average Shrink Loss Across All Stores by Cat')
    
    
cat_plot_shrink()

In [ ]:
def st_plot_sales():
    freq = '20d'
    item_filt = df.groupby(['state', pd.Grouper(key='visit_date', freq=freq)]).mean().reset_index()
    fig = plt.figure(figsize=(40,60))
    i = 1
    count = 0
    for state in item_filt.state.unique():
        count += 1
        if count % 9 == 0:
            i += 1
        foo = item_filt[ item_filt.state == state]
        ax = fig.add_subplot(3,2,i)
        ax.plot(foo.visit_date, foo.sales_value, label=state)
        ax.set_xticklabels(foo.visit_date, rotation=45, ha='right')
        ax.legend()
        ax.set_ylabel('Average Lost Sales/{} ($)'.format(freq))
        ax.set_xlabel('Date')
        ax.grid(alpha=0.3)
        ax.set_title('Average Sales Loss Across Cat by State')
    
st_plot_sales()

In [ ]:
def cust_plot_sales():
    freq = '2w'
    item_filt = df.groupby(['customer_id', pd.Grouper(key='visit_date', freq=freq)]).mean().reset_index()
    fig = plt.figure(figsize=(16,10))
    plt.tight_layout()
    i = 1
    for cust in item_filt.customer_id.unique():
        if cust == '2741156':
            continue
        foo = item_filt[ item_filt.customer_id == cust]
        ax = fig.add_subplot(1,1,1)
        ax.plot(foo.visit_date, foo.sales_value, label=cust)
        #ax.set_xticklabels(foo.visit_date, rotation=45, ha='right')
    plt.legend()
    plt.ylabel('Average Lost Sales/{} ($)'.format(freq))
    plt.xlabel('Date')
    plt.grid(alpha=0.3)
    plt.title('Average Sales Loss Across All Stores by Customer')
    #plt.yscale('log')
    
    
cust_plot_sales()

In [ ]:
plt.figure(figsize=(20,4))
plt.scatter(df.visit_date.values, df.shrink_value.values, alpha=0.2)
plt.yscale('log')
plt.yticks([0.1,1,10])


In [ ]:
df.groupby(['state', pd.Grouper(key='visit_date', freq='w')]).head()

In [ ]:
def st_plot_shrink():
    freq = '20d'
    item_filt = df.groupby(['state', pd.Grouper(key='visit_date', freq=freq)]).mean().reset_index()
    fig = plt.figure(figsize=(40,60))
    i = 1
    count = 0
    for state in item_filt.state.unique():
        count += 1
        if count % 9 == 0:
            i += 1
        foo = item_filt[ item_filt.state == state]
        ax = fig.add_subplot(3,2,i)
        ax.plot(foo.visit_date, foo.qty_shrink, label=state)
        ax.set_xticklabels(foo.visit_date, rotation=45, ha='right')
        ax.legend()
        ax.set_ylabel('Average Shrink/{} ($)'.format(freq))
        ax.set_xlabel('Date')
        ax.grid(alpha=0.3)
        ax.set_title('Average Shrink Loss Across Cat by State')
    
st_plot_shrink()

In [ ]:
crime = pd.read_pickle('data/Crime/crime_clean.pkl')

In [ ]:
def crime_plot():
    freq = 'w'
    item_filt = crime.groupby([pd.Grouper(key='date', freq=freq)]).count().reset_index()
    plt.figure(figsize=(20,8))
    plt.plot(item_filt.date, item_filt.city)
    plt.grid(alpha=0.4)
crime_plot()

In [ ]:
def fit_lstm(X_train, y, batch_size, nb_epoch, neurons):
    X = X_train[]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
        model.reset_states()
    return model

In [ ]:
fit_lstm(X_train, y, None, 1, 20)

# Questions
1. Why are there two salesman ID columns?
2. Which columns are unknown (ie anything inventory out or equivalent)?
3. Target is qty_shrink?
4. What does customer_id represent? It has more values than address1

# Other Data Sources
 - Crime data
 - Food desserts (people that may rely on gas stations for food)
 - Average income
 - Population density

In [ ]:
pd.get_dummies(df, columns=['state'])

In [ ]:
foo = pd.DataFrame()
foo[['a', 'b']] = df[['visit_date', 'address1']]

In [ ]:
foo.head()

# POA
- Create averages:
    - Avg qty shrink/day, shink_sales/day, etc
- Engineer lag terms (ie last visit, last month, last season)
    - Use these in whatever model I want
    - Use the averaged values